## Requirements
- numpy (1.13.0)
- scipy (0.19.0)
- matplotlib (2.0.2)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import robot1 as r1
from util import *
from planning_algorithms import *

D2R = np.pi / 180.0 # conversion constant from deg to rad

In [4]:
# generate three cartesian path in 2D, given in the paper on p. 639
xa, xb, xc = generate_example_paths()

# initial joint configuration for the three paths, also on p. 639
qa_init = np.array([ 0, 20]) * D2R
qb_init = np.array([20, 20]) * D2R
qc_init = np.array([40, 20]) * D2R

In [5]:
# plot initial configuration
plt.figure()

plt.plot(xa[:, 0], xa[:, 1], 'kx')
plt.plot(xb[:, 0], xb[:, 1], 'kx')
plt.plot(xc[:, 0], xc[:, 1], 'kx')

x2, x1 = r1.fk(qa_init, all_links = True)
plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'ko-')
x2, x1 = r1.fk(qb_init, all_links = True)
plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'ko-')
x2, x1 = r1.fk(qc_init, all_links = True)
plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'ko-')

plt.axis([-2, 2, 0, 2.0])
plt.title('Example paths and initial configurations')
plt.xlabel('x', fontsize = 14)
plt.ylabel('y', fontsize = 14)
plt.text(1.7, 0.5, 'A', fontsize = 18)
plt.text(1.6, 1.1, 'B', fontsize = 18)
plt.text(1.4, 1.6, 'C', fontsize = 18)
#plt.show()

## Fig. 6: Apply the four algorithms to path A

In [6]:
qa_exact = exact_path_following(xa, qa_init, r1.fk)

d = 0.5 # maximum deviation for non-exact algorithms

# Taylors algorithm also needs an ik solution for the last point!
#qa_taylor = taylors_algorithm(xa[0], xa[-1], qa_init, qa_exact[-1], d, r1.fk)

qa_opt, xa_opt = local_optimization(xa, qa_init, d, r1.fk, r1.J)

qa_sho, xa_sho = trajectory_shortening(xa, qa_exact, d, r1.fk, angle_max = 175.0 * D2R)

In [9]:
plt.figure(figsize = (6, 4))

plt.subplot(221)
plt.axis([-2, 2, -1, 1])
for i in range(xa.shape[0]):
    x2, x1 = r1.fk(qa_exact[i], all_links = True)
    plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'k-')
plt.plot(xa[:, 0], xa[:, 1], 'ko')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(222)
plt.axis([-2, 2, -1, 1])
for i in range(xa.shape[0]):
    x2, x1 = r1.fk(qa_opt[i], all_links = True)
    plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'k-')
plt.plot(xa[:, 0], xa[:, 1], 'ko')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(223)
plt.axis([-2, 2, -1, 1])
x2, x1 = r1.fk(qa_init, all_links = True)
plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'k-')
plt.plot(xa[:, 0], xa[:, 1], 'ko')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(224)
plt.axis([-2, 2, -1, 1])
for i in range(xa.shape[0]):
    x2, x1 = r1.fk(qa_sho[i], all_links = True)
    plt.plot([0, x1[0], x2[0]], [0, x1[1], x2[1]], 'k-')
plt.plot(xa[:, 0], xa[:, 1], 'ko')
plt.xlabel('x')
plt.ylabel('y')
#plt.show()

In [22]:
from numpy.linalg import norm
delta = 0.1 # acceptable deviation from path
i = 0 # current point looked at (i to i+1)
N = 2 # total number of points in solution

# initial solution
xsol = [xa[0], xa[-1]]
qsol = [qa_init, qa_exact[-1]]

max_iter = 100

#while(i < (N - 1) and max_iter >= 0):
for k in range(5):
    max_iter -= 1
    
    print "Loop " + str(k)
    # interpolate in joint space
    q_mid = (qsol[i] + qsol[i+1]) / 2
    x_mid = r1.fk(q_mid)
    
    # interpolate in cartesian space
    x_mid_cart = (xa[i] + xa[i+1]) / 2
    
    # compare cartesian points
    e = norm(x_mid_cart - x_mid)
    
    print qsol
    print "Error: " + str(e)
    
    if e <= delta:
        # add point to solution
        add_point(xsol, qsol, x_mid, q_mid, i+1)
        N += 1
        i += 2
    else:
        # refine grid with ik solver
        x_ref, q_ref = refine_grid(xsol, qsol, i, r1.fk)
        add_point(xsol, qsol, x_ref, q_ref, i+1)
        N +=1
        
qsol = np.array(qsol)

Loop 0
[array([ 0.        ,  0.34906585]), array([ 2.62836057,  0.66278441])]
Error: 2.43308089779
Loop 1
[array([ 0.        ,  0.34906585]), array([ -9.71423574,  22.33720396]), array([ 2.62836057,  0.66278441])]
Error: 1.11401113524
Loop 2
[array([ 0.        ,  0.34906585]), array([-0.6869834 ,  2.03920619]), array([ -9.71423574,  22.33720396]), array([ 2.62836057,  0.66278441])]
Error: 0.254585697667
Loop 3
[array([ 0.        ,  0.34906585]), array([-0.49006969,  1.43881903]), array([-0.6869834 ,  2.03920619]), array([ -9.71423574,  22.33720396]), array([ 2.62836057,  0.66278441])]
Error: 0.0805785709926
Loop 4
[array([ 0.        ,  0.34906585]), array([-0.24503485,  0.89394244]), array([-0.49006969,  1.43881903]), array([-0.6869834 ,  2.03920619]), array([ -9.71423574,  22.33720396]), array([ 2.62836057,  0.66278441])]
Error: 0.225752153743
